Import di base

In [1]:
from gym import Env
from gym.spaces import Discrete, Box, MultiDiscrete
import copy
from IPython.display import clear_output
import time
import numpy as np
import random
import socket
import math
from scipy.spatial.transform import Rotation as R
import struct

In [12]:
# definizioni variabili globali ===============================================
HOST = "127.0.0.1"
PORT = 7001
PACKET_SIZE = 40

# definizioni funzioni =======================================================
#creo socket e lo faccio partire
def CreaSocket():
    # creo socket server per UE5
    MySocket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

    #Bindo il socket 
    MySocket.bind((HOST,PORT))

    #accendo il socket in ascolto
    MySocket.listen()

    return MySocket

#ciclo di attesa connesione
def AttesaConnesione(MySocket):
    while True:
        RemoteConnection , RemoteAddress = MySocket.accept()
        MySocket.close()
        break
    return RemoteConnection,RemoteAddress

#funzione per sampolare il robot di unreal
def MandaRicevi_Robot(MySocket, R_Angles = [0.0]*5 ):
    """
    Args:
        Socket: socket verso UE5
        R_XXX: angoli da applicare ai bracci del robot, in rad
    Return:
        Target_Coor: X Y Z
        EE_Coor: X Y Z
        EE_Rotator: X Y Z W
    - Converte angoli da rad a gradi
    - crea il messaggio binario
    - manda e riceve
    - scompatta il messaggio binario di ritorno in un buffer
    - separa il buffer negli oggetti ricevuti
    """
    Angoli = copy.deepcopy(R_Angles) # questo perchè python è cattivo e le liste le porta in giro come ref
    
    #converto angoli in gradi
    for i in range(5):
        Angoli[i] = math.degrees(Angoli[i] * math.pi)

    # preparo messaggio binario
    Messaggio =    (struct.pack("f",Angoli[0])+
                    struct.pack("f",Angoli[1])+
                    struct.pack("f",Angoli[2])+
                    struct.pack("f",Angoli[3])+
                    struct.pack("f",Angoli[4])+
                    struct.pack("f",0.0)+
                    struct.pack("f",0.0) # questo è un "flag" di comandi. 0: comando angoli, 1: comando reset target
                    )

    # mando e ricevo
    MySocket.sendall(Messaggio)
    ReceivedData = MySocket.recv(PACKET_SIZE)

    ParsedData = [] # lista dei valori float ricevuti parsati
    for i in range(0,40,4):
        ParsedData.append( struct.unpack("f",ReceivedData[i:i+4])[0])
    
    # arrivati: Target Coordinate, EE Coordinate, EE Rotator Quaternion
    Target_Coor = ParsedData[0:3]
    EE_Coor     = ParsedData[3:6]
    EE_Rotator  = ParsedData[6:10]

    return Target_Coor, EE_Coor, EE_Rotator

# classe environment di gioco =============================================

class RobotEnv(Env):
    def __init__(self):
        # azioni che possiamo fare: 5 angoli normalizzati
        self.action_space = Box(low=-1.0,high=1.0,shape=(5,))
        self.last_action = [0.0] * 5

        # Observation space: Target_Coor (3), Effector Coor (3), Effector to Target Polar (3), angoli Attuali (5), velocità (5), last action (5)
        self.observation_space = Box(low=-10000,high=10000,shape=(24,))
        self.state = [0.0] * 24
        
        # Randomizzo angoli
        for i in range(10,15):
            self.state[i] = random.uniform( -1.0 , +1.0)

        # buffers
        self.BufferAngoliAttuali = self.state[10:15]
        self.BufferSpeed = [0.0]*5
        

        # lunghezza esercizio
        self.Exercise_length = 1000

    def step(self, action):
        global UE5_Socket
        done = False
        # applichiamo azione 
        # velocità discretizzata 0-1.5 gradi/step ( 0.0261799 rad/step )
        # prendo il nuovo target proposto da Action, e calcolo un aumento in quella direzione di un grado per step
        # se Action corrisponde ad angolo attuale, velocità passa a zero
        
        #DeNorm_Action = list(action * (math.pi))

        NuoviAngoli = [0.0]*5

        for i in range(5):

            if self.BufferAngoliAttuali[i] < action[i]:
                
                NuoviAngoli[i] = self.BufferAngoliAttuali[i] + math.radians(0.5)
                self.BufferSpeed[i] = +0.0261799

            elif self.BufferAngoliAttuali[i] > action[i]:

                NuoviAngoli[i] = self.BufferAngoliAttuali[i] - math.radians(0.5)
                self.BufferSpeed[i] = -0.0261799

            else:
                self.BufferSpeed[i] = 0

        self.BufferAngoliAttuali = NuoviAngoli
        # fine applicazione
           
        # Mando ricevo!
        Target_Coor, EE_Coor, EE_Rotator = MandaRicevi_Robot(UE5_Socket,R_Angles= NuoviAngoli)
        
            
        # creo polari 
        EE_to_Base_Rotation = R.from_quat(EE_Rotator)
        Base_to_EE_Rotation = EE_to_Base_Rotation.inv()
        # to invert the homogeneous transformation
        # R' = R^-1
        Base_to_EE_Quat = Base_to_EE_Rotation.as_quat()
        # t' = - R^-1 * t
        Base_to_EE_Translation = - Base_to_EE_Rotation.apply(EE_Coor)

        EE_to_Target_Coord = change_reference_frame(Target_Coor,Base_to_EE_Translation,Base_to_EE_Quat)
        EE_to_Target_Polar = cartesian_to_polar_3d(EE_to_Target_Coord)


        # compilo lo state
        self.state[0:3]   = Target_Coor
        self.state[3:6]   = EE_Coor
        self.state[6:9]   = EE_to_Target_Polar

        self.state[9:14]  = self.BufferAngoliAttuali
        self.state[14:19] = self.BufferSpeed

        self.state[19:24] = self.last_action 
        
        # mi segno attuale azione, per futuro
        self.last_action = action

        # riduzione tempo 
        self.Exercise_length -= 1

        
        
        #calcolo reward 
        reward = 0

        # reward base:
        reward += -0.005 * EE_to_Target_Polar[0]

        # reward aggiuntivo:
        if EE_to_Target_Polar[0] <= 30:
            reward = 2
            done = True     
        
        
        #verfica se tempo finito
        if done == False:  # solo se non ho già finito per altri motivi
            if self.Exercise_length <= 0:
                done = True
            else:
                done = False
                     
        # segnaposto per info
        info = {}
        # ritorniamo informazioin sullo step
        return np.array(self.state), reward, done, info

    def render(self,mode=0):
        print("Tempo: "+str(self.Exercise_length) + "-- state: " + str(self.state))

    def reset(self):
        print("RESETTTT")
        # mando segnale ad UE5 di resettare il target:
        # preparo messaggio binario
        """
        Messaggio =(struct.pack("f",0.0)+
                    struct.pack("f",0.0)+
                    struct.pack("f",0.0)+
                    struct.pack("f",0.0)+
                    struct.pack("f",0.0)+
                    struct.pack("f",0.0)+
                    struct.pack("f",1.0)
                    )
        
        # mando
        UE5_Socket.sendall(Messaggio)
        """

        #resettiamo 
        self.last_action = [0.0] * 5
        self.state = [0.0] * 24

        # Randomizzo angoli
        for i in range(10,15):
            self.state[i] = random.uniform( -1.0 , +1.0)

        # buffers
        self.BufferAngoliAttuali = self.state[10:15]
        self.BufferSpeed = [0.0]*5

        # resettiamo tempo
        self.Exercise_length = 1000

        return np.array(self.state)

def change_reference_frame(point, translation, quaternion):
    """Transform a point from one reference frame to another, given
        the translation vector between the two frames and the quaternion
        between  the two frames.

    Args:
        point (array_like,shape(3,) or shape(N,3)): x,y,z coordinates of the point in the original frame
        translation (array_like,shape(3,)): translation vector from the original frame to the new frame 
        quaternion (array_like,shape(4,)): quaternion from the original frame to the new frame

    Returns:
        ndarray,shape(3,): x,y,z coordinates of the point in the new frame.
        
    """

    #point = [1,2,3]
    #point = np.array([1,2,3])
    #point = np.array([[11,12,13],[21,22,23]]) # point.shape = (2,3) # point (11,12,13)  and point (21,22,23)

    # Apply rotation
    r = R.from_quat(quaternion)
    rotated_point = r.apply(np.array(point))
    # Apply translation
    translated_point = np.add(rotated_point, np.array(translation))

    return translated_point

def cartesian_to_polar_3d(cartesian_coordinates):
    """Transform 3D cartesian coordinates to 3D polar coordinates.

    Args:
        cartesian_coordinates (list): [x,y,z] coordinates of target point.

    Returns:
        list: [r,phi,theta] polar coordinates of point.

    """

    x = cartesian_coordinates[0]
    y = cartesian_coordinates[1]
    z = cartesian_coordinates[2]
    r =  np.sqrt(x**2+y**2+z**2)
    #? phi is defined in [-pi, +pi]
    phi = np.arctan2(y,x)
    #? theta is defined in [0, +pi]
    theta = np.arccos(z/r)

    return [r,theta,phi]

# codice ==================================================================

#creo e ascolto
Server_Socket = CreaSocket()

#attendo e connetto con UE5 
UE5_Socket , UE5_Address = AttesaConnesione(Server_Socket)

#confermo connesione aperta
print(f"Aperto socket con UE5: {UE5_Address}")

SimpleRobotEnv = RobotEnv()


Aperto socket con UE5: ('127.0.0.1', 64500)


In [3]:
from stable_baselines3.common.env_checker import check_env
check_env(SimpleRobotEnv)

RESETTTT


In [13]:
# giro di prova dell'exercise
episodes = 1
for episode in range(1, episodes+1):
    state = SimpleRobotEnv.reset()
    done = False
    score = 0 
    
    while not done:
        SimpleRobotEnv.render(mode=0)
        action = SimpleRobotEnv.action_space.sample()
        n_state, reward, done, info = SimpleRobotEnv.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

RESETTTT
Tempo: 1000-- state: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.5294440224451702, 0.4575916932934587, 0.6812629558553911, 0.38171055191416947, 0.4223519827991873, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Tempo: 999-- state: [-47.13887023925781, -74.34370422363281, 28.943344116210938, -5.3857269287109375, -31.06472396850586, -43.57876968383789, 94.21174076654125, 2.4573929492246345, 1.1711379985298183, -0.5381706687051419, 0.44886504703348706, 0.6725363095954194, 0.3729839056541978, 0.4136253365392156, -0.0261799, -0.0261799, -0.0261799, -0.0261799, -0.0261799, 0.0, 0.0, 0.0, 0.0, 0.0]
Tempo: 998-- state: [-47.13887023925781, -74.34370422363281, 28.943344116210938, -5.810874938964844, -34.709747314453125, -43.7162971496582, 92.51095721429188, 2.4067850718562713, 1.110642123912557, -0.5294440224451702, 0.4401384007735154, 0.6638096633354478, 0.36425725939422615, 0.40489869027924397, 0.0261799, -0.0261799, -0.0261799, -0.0261799, -0.0261799, -0.624998, 0.07162835, 

In [15]:
from stable_baselines3 import TD3
from stable_baselines3.td3.policies import MlpPolicy
from stable_baselines3.common.noise import NormalActionNoise

n_actions = SimpleRobotEnv.action_space.shape[-1]
action_noise = NormalActionNoise( mean=np.zeros(n_actions) , sigma=2 * np.ones(n_actions) )

model = TD3(MlpPolicy, SimpleRobotEnv, action_noise=action_noise, verbose=1, learning_rate=0.001, buffer_size=1000000, batch_size=256,gamma=0.99,seed=123
            ,train_freq=(1,"episode"), tensorboard_log="./TD3_Madness_TB/")

model.learn( log_interval = 10 , total_timesteps = 2500000 )

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
RESETTTT
Logging to ./TD3_Madness_TB/TD3_1
RESETTTT
RESETTTT
RESETTTT


Allenamento approfondito:

In [7]:
model.learn(total_timesteps=2500000, log_interval=4)

RESETTTT
RESETTTT
RESETTTT
RESETTTT
RESETTTT
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 500      |
|    ep_rew_mean     | -189     |
| time/              |          |
|    episodes        | 4        |
|    fps             | 60       |
|    time_elapsed    | 33       |
|    total_timesteps | 2000     |
| train/             |          |
|    actor_loss      | 60       |
|    critic_loss     | 20.4     |
|    learning_rate   | 0.001    |
|    n_updates       | 251845   |
---------------------------------
RESETTTT
RESETTTT
RESETTTT
RESETTTT
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 500      |
|    ep_rew_mean     | -202     |
| time/              |          |
|    episodes        | 8        |
|    fps             | 55       |
|    time_elapsed    | 72       |
|    total_timesteps | 4000     |
| train/             |          |
|    actor_loss      | 59.9     |
|    critic_loss     | 21.7     |
|

ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

Piccolo test del modello:

In [43]:
obs = SimpleRobotEnv.reset()

while True:
    action, _states = model.predict(obs)
    obs, rewards, dones, info = SimpleRobotEnv.step(action)
    SimpleRobotEnv.render()

RESETTTT
Tempo: 499-- state: [-47.13887023925781, -74.34370422363281, 28.943344116210938, 10.55939769744873, 40.570640563964844, 15.000768661499023, 129.33983211358048, 0.5180013279643083, -2.494426885718058, 0.56653299512035, 0.1503837703234686, 0.6748412301541976, 0.4269685856757651, -0.8438180374253567, 0.0261799, -0.0261799, -0.0261799, -0.0261799, 0.0261799, 0.0, 0.0, 0.0, 0.0, 0.0]
Tempo: 498-- state: [-47.13887023925781, -74.34370422363281, 28.943344116210938, 9.286185264587402, 42.53335952758789, 16.106307983398438, 130.41788380584183, 0.5003789342412328, -2.342027620385858, 0.5578063488603784, 0.14165712406349695, 0.666114583894226, 0.41824193941579346, -0.8525446836853283, -0.0261799, -0.0261799, -0.0261799, -0.0261799, -0.0261799, 0.99124336, -0.99189305, -0.9999886, -0.64627504, 0.17370248]
Tempo: 497-- state: [-47.13887023925781, -74.34370422363281, 28.943344116210938, 7.913341522216797, 44.463748931884766, 17.376785278320312, 131.45243333039195, 0.49244510483729625, -2.18

KeyboardInterrupt: 

Chiusure varie

In [ ]:
# spengo socket prima di uscire
UE5_Socket.shutdown(socket.SHUT_RDWR)
UE5_Socket.close()